<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> pySIRUS </h1></center>

Notebbok de test de pysirus avec une RF.


In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import time

import numpy as np

from sklearn.tree import _tree, export_text,plot_tree
from pysirus.models.basic import Node,SirusDTreeClassifier

In [ ]:
print(os.getcwd())

In [ ]:
os.path.join(os.getcwd(), os.pardir)

In [ ]:
os.pardir

In [ ]:
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_train.shape

In [ ]:
np.quantile(X,q=0.1,axis=0) # puis np.clip avec la liste des quantiles. 
#On replace les valeurs de chaque features par ça avec le clip. la RF est ensuite apprise sur ça

In [ ]:
quantile =0.1
print(int(100//(quantile*100)))
list_quantile = [np.quantile(X_train,q=i*quantile,axis=0) for i in range(int(100//(quantile*100))+1)]
list_quantile

In [ ]:
array_quantile = np.array(list_quantile )
array_quantile

In [ ]:
X_train_2 = X_train.copy()
#X_train_2[:,0] = array_quantile[out,0]
#X_train_2[:,0]

In [ ]:
for dim in range(X_train.shape[1]):
    out = np.searchsorted(array_quantile[:,dim], X_train[:,dim],side='left')
    X_train_2[:,dim] = array_quantile[out,dim]

In [ ]:
X_train_2

In [ ]:
X_train

In [ ]:
X_train_original = X_train

In [ ]:
X_train = X_train_2

In [ ]:
X_train

In [ ]:
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
tree.plot_tree(clf)

## Essai .fit() :

In [ ]:
rule = export_text(clf,feature_names=iris.feature_names,show_weights=True,max_depth=10)
rule

In [ ]:
rule[1]

In [ ]:
print(rule)

In [ ]:
tree = clf.tree_
iterator = enumerate(zip(tree.children_left, tree.children_right, tree.feature, tree.threshold, tree.value))
for node_idx, data in iterator:
    left, right, feature, th, value = data

    # left: index of left child (if any)
    # right: index of right child (if any)
    # feature: index of the feature to check
    # th: the threshold to compare against
    # value: values associated with classes            

    # for classifier, value is 0 except the index of the class to return
    print(value)
    class_idx = np.argmax(value[0])

    if left == -1 and right == -1:
        print('{} LEAF: return class={}'.format(node_idx, class_idx))
    else:
        print('{} NODE: if feature[{}] < {} then next={} else next={}'.format(node_idx, feature, th, left, right)) 

In [ ]:
(5,2,'R') == (5,2,'R')

In [ ]:
tree.children_left[0]

In [ ]:
tree.max_depth

## Functions :

In [ ]:
def explore_tree(tree,node_id,side):
    if tree.children_left[node_id] != _tree.TREE_LEAF: # possible to add a max_depth constraint exploration value
        id_left_child = tree.children_left[node_id]
        id_right_child = tree.children_right[node_id]
        #children = [
        #        (tree.feature[id_left_child],tree.threshold[id_left_child],'L'),
        #        (tree.feature[id_right_child],tree.threshold[id_right_child],'R'),
        #        ]
        children = [
            explore_tree(tree,id_left_child,'L'),
            explore_tree(tree,id_right_child,'R')
        ]
    else:
        return Node(feature=tree.feature[node_id],treshold=tree.threshold[node_id],side=side,node_id=node_id)
    
    return Node(tree.feature[node_id],tree.threshold[node_id],side,node_id,*children)

In [ ]:
res = explore_tree(tree=tree,node_id=0,side='Root')


In [ ]:
res.children

In [ ]:
res.children[0].feature



In [ ]:
tree_structure = [[]]
stack = [(res,0)]  # start with the root node id (0) and its depth (0)
depth = 0
while len(stack) > 0:
    curr_rule,indice_in_tree_struct = stack.pop()
    is_split_node = (curr_rule.feature != -2)

    if is_split_node:
        rule_left = (curr_rule.feature,curr_rule.treshold,'L')
        rule_right = (curr_rule.feature,curr_rule.treshold,'R')
        common_path_rules = tree_structure[indice_in_tree_struct].copy()
        common_path_rules.append(rule_right)
        tree_structure.append(common_path_rules) ## DROITE : Ajouté à la fin
        tree_structure[indice_in_tree_struct].append(rule_left) ## GAUCHE  : On le rajoute selon indice_in_tree_struct

        stack.append((curr_rule.children[0],indice_in_tree_struct))
        stack.append(( curr_rule.children[1],len(tree_structure)-1 ))
    else:
        #print('c')
        continue

In [ ]:
tree_structure

In [ ]:
len(tree_structure)

In [ ]:
all_paths_list = []
for i in range(len(tree_structure)):
    #print('*****'*10)
    #print(tree_structure[i])
    max_size_curr_path = len(tree_structure[i])
    #for k in range()
    for j in range(max_size_curr_path): ## fonction
        #print('####')
        #print('curr sous path :', tree_structure[i][:(max_size_curr_path-j)])
        all_paths_list.append(tree_structure[i][:(max_size_curr_path-j)])

In [ ]:
all_paths_list

In [ ]:
def split_sub_rules(path,is_removing_singleton=False):
    list_sub_path = []
    max_size_curr_path = len(path)
    if is_removing_singleton:
        int_to_add = 1
    else:
        int_to_add = 0
    for j in range(max_size_curr_path-int_to_add):
        list_sub_path.append(path[:(max_size_curr_path-j)])
    return list_sub_path


all_paths_list = []
for i in range(len(tree_structure)):
    print('*****'*10)
    curr_path = tree_structure[i]
    max_size_curr_path = len(curr_path)
    
    for k in range(max_size_curr_path):
        list_sub_path = split_sub_rules(curr_path[k:],is_removing_singleton=False)
        print('ici : ', list_sub_path)
        all_paths_list.extend(list_sub_path)

    ## CAS plus complexes :
    if max_size_curr_path ==1:
        #all_paths_list.append(curr_path)
        print("SKIIIIP")
        continue
    else:
        curr_path_size_pair = ( (max_size_curr_path%2)==0)
        print('TAIIIILLE : ',max_size_curr_path)

        if curr_path_size_pair:
            print('aa')
            #list_sub_path = split_sub_rules(curr_path)
            #all_paths_list.extend(list_sub_path)          
            for k in range(1, (max_size_curr_path//2) ):
                print('curr path : ',curr_path[k:max_size_curr_path-k])
                list_sub_path = split_sub_rules(curr_path[k:max_size_curr_path-k],is_removing_singleton=True)
                print('list_sub_path : ',list_sub_path)
                all_paths_list.extend(list_sub_path)
        else:
            print('b')
            #list_sub_path = split_sub_rules(curr_path)
            #all_paths_list.extend(list_sub_path)
            for k in range(1,(max_size_curr_path//2) ):
                print('curr path : ',curr_path[k:max_size_curr_path-k])
                list_sub_path = split_sub_rules(curr_path[k:max_size_curr_path-k],is_removing_singleton=True)
                all_paths_list.extend(list_sub_path)
                print('list_sub_path : ',list_sub_path)
                if k == (max_size_curr_path//2): #case odd last 
                    list_sub_path = split_sub_rules(curr_path[k:max_size_curr_path-(k-1)],is_removing_singleton=True)
                    print('list_sub_path : ',list_sub_path)
                    all_paths_list.extend(list_sub_path)
    print('end iteration all_paths_list : ',all_paths_list)
        

In [ ]:
len(tree_structure)

In [ ]:
3 // 2

In [ ]:
tree_structure[1]

In [ ]:
tree_structure[1][1:-1]

In [ ]:
all_paths_list

In [ ]:
len(all_paths_list)

In [ ]:
print(len(tree_structure[0]),len(tree_structure[1]),len(tree_structure[2]),len(tree_structure[3]),len(tree_structure[4]),
     len(tree_structure[5]),len(tree_structure[6]))

In [ ]:
import math

In [ ]:
1 + (3*4)/2 + (4*5/2)+1 + 3*4/2 + (4*5/2)+1 + (4*5/2)+1 + (4*5/2)+1

In [ ]:
1 + 6 + 11 + 6 + 11 + 11 + 11

In [ ]:
all_paths_list_str = [str(elem) for elem in all_paths_list]

In [ ]:
unique_str_rules,indices_rules,count_rules = np.unique(all_paths_list_str,return_counts=True,return_index=True)

In [ ]:
unique_str_rules

In [ ]:
len(unique_str_rules)

In [ ]:
count_rules

In [ ]:
-np.sort(-(count_rules / len(count_rules))) > 0.09

In [ ]:
(-np.sort(-(count_rules / len(count_rules))) > 0.09).sum()

In [ ]:
len(count_rules)

In [ ]:
count_rules[23]

In [ ]:
indices_rules

In [ ]:
np.sort(-count_rules)

In [ ]:
count_sort_ind = np.argsort(-count_rules)
count_sort_ind

In [ ]:
count_sort_ind[:10]

In [ ]:
count_sort_ind[:1]

In [ ]:
all_paths_list[47]

In [ ]:
all_paths_list[81]

selected_rules_indices = count_sort_ind[:10]
selected_rules = [all_paths_list[i] for i in selected_rules_indices] ## pas utile ici. Le faire en bas.
selected_rules

In [ ]:
def from_rules_to_constraint(rule):
    dimension = rule[0]
    treshold = rule[1]
    sign = rule[2]
    return dimension,treshold,sign

In [ ]:
dimension,treshold,sign = from_rules_to_constraint(rule=all_paths_list[47][0])
dimension,treshold,sign

In [ ]:
(X_train[:,dimension]<=treshold)

In [ ]:
(X_train[:,dimension]<=treshold) & (X_train[:,dimension]<=treshold-1)

In [ ]:
((X_train[:,dimension]<=treshold) & (X_train[:,dimension]<=treshold-1)).shape

In [ ]:
X_train.shape

In [ ]:
X_train[(X_train[:,dimension]<=treshold) & (X_train[:,dimension]<=treshold-1)]

In [ ]:
def generate_single_rule_mask(X,dimension,treshold,sign):
    if sign=='L':
        return (X[:,dimension]<=treshold) #.mean()
    else:
        return (X[:,dimension]>treshold)#.mean()


In [ ]:
from operator import and_
from functools import reduce
from collections import Counter
list_mask_by_rules = []
for indice in count_sort_ind[:10]:#for loop for getting all the values in train passing the rules
    current_rules = all_paths_list[indice]
    list_mask=[]
    for j in range(len(current_rules)):
        print(current_rules)
        print(current_rules[j])
        dimension,treshold,sign = from_rules_to_constraint(rule=current_rules[j])
        mask = generate_single_rule_mask(X=X_train,dimension=dimension,treshold=treshold,sign=sign)
        list_mask.append(mask)
    final_mask = reduce(and_, list_mask)
    list_mask_by_rules.append(final_mask)
    print("########")
    #print((y_train[final_mask]).sum() / len(y_train[final_mask])) #binary case
    n_classe=3
    y_train_rule = y_train[final_mask]
    print(y_train_rule)
    for cl in range(n_classe):# nb de classe
        print('Class ',cl,' : ',len(y_train_rule[y_train_rule==cl]) / len(y_train_rule)) #binary case
    print(Counter(y_train_rule))
    print(Counter(y_train_rule).most_common(1)[0][0])
    

In [ ]:
np.unique(y_train)

In [ ]:
np.mean(list_mask_by_rules,axis=1)

In [ ]:
np.mean(list_mask_by_rules,axis=1).argmax()

In [ ]:
np.mean(list_mask_by_rules,axis=1)

In [ ]:
y_train.dtype

In [ ]:
y_train.dtype == int

In [ ]:
all_paths_list[:10]

In [ ]:
def get_probas_rules(X,rules):
    list_mask=[]
    for j in range(len(rules)):
        dimension,treshold,sign = from_rules_to_constraint(rule=rules[j])
        mask = generate_single_rule_mask(X=X,dimension=dimension,treshold=treshold,sign=sign)
        list_mask.append(mask)
    final_mask = reduce(and_, list_mask)
    return final_mask
    

In [ ]:
aa = [2,3,4]
bb = [4,6,8]

In [ ]:
cc = np.array([np.random.uniform(low=aa[i],high=bb[i],size=(10)) for i in range(len(aa))]).T
cc

In [ ]:
cc.T

In [ ]:
cc .shape

In [ ]:
probas = np.zeros(len(all_paths_list[:10]))
for i,rules in enumerate(all_paths_list[:10]):
    mask = get_probas_rules(X=X_train,rules=rules)
    X_train_valid = X_train[mask]
    probas[i] = len(X_train_valid) / len(X_train)
probas    

In [ ]:
all_paths_list[0]

In [ ]:
all_paths_list[1]

In [ ]:
aa= [1,2,3,4,5,6]
aa[3:8]

In [ ]:
all_paths_list[0]+all_paths_list[1]

In [ ]:
if not False:
    print('a')

## Class : 

### Dtree :

In [ ]:
from sklearn.metrics import average_precision_score, accuracy_score,roc_auc_score

In [ ]:
from pysirus.models._QuantileSplitter import QuantileBestSplitter

In [ ]:
X_train_original

In [ ]:
clf_dtree = DecisionTreeClassifier(max_depth=10, random_state=0)
clf_dtree.fit(X_train_original, y_train)
y_pred_dtree = clf_dtree.predict(X_test)
y_pred_proba_dtree = clf_dtree.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))

In [ ]:
STree = SirusDTreeClassifier(max_depth=10, random_state=0)
#unique_str_rules_test,indices_rules_test,count_rules_test = STree.fit(X_train_original,y_train)
STree.fit(X_train_original,y_train,quantile=10)

In [ ]:
STree.print_rules(max_rules=10)

In [ ]:
y_pred_sirus = STree.predict(X_test)
y_pred_proba_sirus = STree.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))

In [ ]:
STree = SirusDTreeClassifier(max_depth=10, random_state=0,splitter="quantile")
#unique_str_rules_test,indices_rules_test,count_rules_test = STree.fit(X_train_original,y_train)
STree.fit(X_train_original,y_train,quantile=10)
y_pred_sirus = STree.predict(X_test)
y_pred_proba_sirus = STree.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))

In [ ]:
STree.print_rules(max_rules=10)

In [ ]:
from sklearn.tree._splitter import Splitter

In [ ]:
from sklearn.tree._criterion import Criterion


In [ ]:
splitter = Splitter(Criterion('gini'),2,3,4,5,None)

In [ ]:
isinstance(splitter, Splitter)

In [ ]:
splitterAbd = QuantileBestSplitter(Criterion('gini'),2,3,4,5,None)

In [ ]:
isinstance(splitterAbd, Splitter)

### Ensemble : 

In [ ]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [ ]:
from pysirus.models.basic import SirusRFClassifier,SirusGBClassifier

In [ ]:
clf_rf = RandomForestClassifier(max_depth=10, random_state=0)
clf_rf.fit(X_train_original, y_train)
y_pred_dtree = clf_rf.predict(X_test)
y_pred_proba_dtree = clf_rf.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))

In [ ]:
clf_gb = GradientBoostingClassifier(random_state=0)
clf_gb.fit(X_train_original, y_train)
y_pred_dtree = clf_gb.predict(X_test)
y_pred_proba_dtree = clf_gb.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))

In [ ]:
RFSirus = SirusRFClassifier(max_depth=10, random_state=0,splitter="quantile")
start = time.time()
RFSirus.fit(X_train_original,y_train,quantile=10,batch_size_post_treatment=50,p0=0.01)
end = time.time()
y_pred_sirus = RFSirus.predict(X_test)
y_pred_proba_sirus = RFSirus.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))
time_2 = end-start
print('Fitting time = ',time_2 ,'s')

In [ ]:
RFSirus.n_rules

In [ ]:
RFSirus.print_rules(max_rules=10)

In [ ]:
GBSirus = SirusGBClassifier(max_depth=10, random_state=0,splitter="quantile")
start = time.time()
GBSirus.fit(X_train_original,y_train,quantile=10,batch_size_post_treatment=50,p0=0.01)
end = time.time()
y_pred_sirus = GBSirus.predict(X_test)
y_pred_proba_sirus = GBSirus.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))
print('GBSirus.n_rules : ',GBSirus.n_rules)
time_4 = end-start
print('Fitting time = ',time_4 ,'s')

In [ ]:
GBSirus.print_rules(max_rules=10)

In [ ]:
GBSirus.n_rules

In [ ]:
y_pred_proba_sirus.sum(axis=1)

In [ ]:
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))

In [ ]:
print('a')